In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import operator
import gc
import time

import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
import string
import re

import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [2]:
N = 500

In [3]:
train = pd.read_csv('./data/train_tweets.csv')
dev = pd.read_csv('./data/dev_tweets.csv')
test = pd.read_csv('./data/test_tweets.csv')

train['tweet'] = train['tweet'].astype(str)
dev['tweet'] = dev['tweet'].astype(str)
test['tweet'] = test['tweet'].astype(str)

In [4]:
print('Train shape:', train.shape)
print('Dev shape  :', dev.shape)
print('Test shape :', test.shape)

Train shape: (96585, 4)
Dev shape  : (34028, 4)
Test shape : (32977, 4)


In [5]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for special in specials:
        text = text.replace(special, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(' ')])
    return text

In [6]:
# remove '@USER_*****'
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))
dev['tweet'] = dev['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))

# clean contruction
train['tweet'] = train['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
dev['tweet'] = dev['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['tweet'] = test['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))

# lower text
train['tweet'] = train['tweet'].apply(lambda x: x.lower())
dev['tweet'] = dev['tweet'].apply(lambda x: x.lower())
test['tweet'] = test['tweet'].apply(lambda x: x.lower())

# remove stop words
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))
dev['tweet'] = dev['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))

# remove non alphabetic chars
train['tweet'] = train['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))
dev['tweet'] = dev['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))
test['tweet'] = test['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))

# strip text
train['tweet'] = train['tweet'].apply(lambda x: x.strip())
dev['tweet'] = dev['tweet'].apply(lambda x: x.strip())
test['tweet'] = test['tweet'].apply(lambda x: x.strip())

In [7]:
def merge_same_user(df):
    df_class = df[['user-id', 'class']].groupby('user-id').apply(lambda x: x['class'].mode()).reset_index()
    df_class = df_class.rename(columns={0: 'class'})
    df_new = df[['user-id', 'tweet']].groupby('user-id').agg(lambda x: ' '.join(x))
    df_new = pd.merge(df_new, df_class, on='user-id')
    
    return df_new

In [8]:
train_per_user = merge_same_user(train)
dev_per_user = merge_same_user(dev)
test_per_user = merge_same_user(test)

train_per_user = pd.concat([train_per_user, dev_per_user], axis=0)

In [9]:
print('Train per user shape:', train_per_user.shape)
print('Test per user shape :', test_per_user.shape)

Train per user shape: (3190, 3)
Test per user shape : (802, 3)


In [10]:
word_list = np.load(f'best{N}.npy')

vectorizer = CountVectorizer(vocabulary=word_list)
vectorizer.fit(train_per_user['tweet'])

train_vec = vectorizer.transform(train_per_user['tweet'])
test_vec = vectorizer.transform(test_per_user['tweet'])

train_vec = pd.DataFrame(train_vec.todense(), columns=vectorizer.get_feature_names())
test_vec = pd.DataFrame(test_vec.todense(), columns=vectorizer.get_feature_names())

train_per_user = train_per_user.drop(['tweet'], axis=1)
test_per_user = test_per_user.drop(['tweet'], axis=1)

train_per_user = train_per_user.reset_index(drop=True)
test_per_user = test_per_user.reset_index(drop=True)
train_vec = train_vec.reset_index(drop=True)
test_vec = test_vec.reset_index(drop=True)

train_per_user = pd.concat([train_per_user, train_vec], axis=1)
test_per_user = pd.concat([test_per_user, test_vec], axis=1)

In [11]:
print('Train per user shape:', train_per_user.shape)
print('Test per user shape :', test_per_user.shape)

Train per user shape: (3190, 972)
Test per user shape : (802, 972)


In [14]:
def accuracy(data, pred):
    y_true = data
    reshaped_pred = pred.reshape(3, len(pred) // 3)
    y_pred = np.argmax(reshaped_pred, axis=0)
    acc = np.mean(y_true == y_pred)
    return 'accuracy', acc, True

def train_model(X, y, X_test, params, fold_num):
    result_dict = {}
    oof = np.zeros((len(X), 3))
    prediction = np.zeros((len(X_test), 3))
    scores = []
    features = X.columns
    feature_importance = pd.DataFrame()

    folds = StratifiedKFold(n_splits=fold_num)
    
    for fold, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        print(f'\nFold {fold + 1} started at {time.ctime()}')
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        model = lgb.LGBMClassifier(
            **params,
            )
        
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric=accuracy,
                  verbose=False,
                  early_stopping_rounds=100)
        
        y_pred_val_proba = model.predict_proba(X_val)
        y_pred_val = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_test)
        
        accuracy_val = accuracy_score(y_val, y_pred_val)
        scores.append(accuracy_val)
        
        print('\nFold {0: d} accuracy: {1: .4f}'.format(fold + 1, accuracy_val))
        
        oof[val_idx] += y_pred_val_proba.reshape(-1, 3)
        prediction += y_pred_proba / fold_num
        
        fold_importance = pd.DataFrame()
        fold_importance['feature'] = features
        fold_importance['importance'] = model.feature_importances_
        fold_importance['fold'] = fold + 1
        feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
        
    print('\nCV mean accuracy {0: .4f}'.format(np.mean(scores)))
    
    cols = feature_importance[['feature', 'importance']].groupby('feature').mean().sort_values(by='importance', ascending=False).index
    best_features = feature_importance[['feature', 'importance']].groupby('feature').mean().sort_values(by='importance', ascending=False).reset_index()

    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    result_dict['feature_importance'] = best_features
    
#     plt.figure(figsize=(16,128))
#     sns.barplot(x='importance', y='feature', data=best_features)
#     plt.title('LGB Features (avg over folds)')
        
    return result_dict

In [15]:
X = train_per_user.drop(['user-id', 'class'], axis=1)
X_test = test_per_user.drop(['user-id', 'class'], axis=1)

y = train_per_user['class']
encoder = LabelEncoder()
y = encoder.fit_transform(y)


params = {
    'n_estimators': 5000,
}

result_dict = train_model(X, y, X_test, params, 10)


Fold 1 started at Sun Oct 13 09:49:38 2019

Fold  1 accuracy:  0.8287

Fold 2 started at Sun Oct 13 09:49:51 2019

Fold  2 accuracy:  0.8349

Fold 3 started at Sun Oct 13 09:50:03 2019

Fold  3 accuracy:  0.8094

Fold 4 started at Sun Oct 13 09:50:15 2019

Fold  4 accuracy:  0.8219

Fold 5 started at Sun Oct 13 09:50:30 2019

Fold  5 accuracy:  0.8428

Fold 6 started at Sun Oct 13 09:50:43 2019

Fold  6 accuracy:  0.8239

Fold 7 started at Sun Oct 13 09:50:53 2019

Fold  7 accuracy:  0.8019

Fold 8 started at Sun Oct 13 09:51:09 2019

Fold  8 accuracy:  0.8302

Fold 9 started at Sun Oct 13 09:51:22 2019

Fold  9 accuracy:  0.8459

Fold 10 started at Sun Oct 13 09:51:32 2019

Fold  10 accuracy:  0.8365

CV mean accuracy  0.8276


In [16]:
validation = train_per_user[['user-id', 'class']]
validation['prediction'] = encoder.inverse_transform(np.argmax(result_dict['oof'], axis=1))
validation[encoder.classes_[0]] = result_dict['oof'][:, 0]
validation[encoder.classes_[1]] = result_dict['oof'][:, 1]
validation[encoder.classes_[2]] = result_dict['oof'][:, 2]
validation.to_csv(f'./result-per-user/lgbm-best{N}-val.csv', index=False)
accuracy_score(validation['class'], validation['prediction'])

0.8275862068965517

In [17]:
submission = test_per_user[['user-id', 'class']]
submission['prediction'] = encoder.inverse_transform(np.argmax(result_dict['prediction'], axis=1))
submission[encoder.classes_[0]] = result_dict['prediction'][:, 0]
submission[encoder.classes_[1]] = result_dict['prediction'][:, 1]
submission[encoder.classes_[2]] = result_dict['prediction'][:, 2]
submission.to_csv(f'./result-per-user/lgbm-best{N}-sub.csv', index=False)